In [9]:
import numpy as np
import pandas as pd
import glob

import time
import datetime as dt
from datetime import date
from datetime import datetime

import re
  
# DEFINE HELPER FUNCTIONS
# CLEAN DATA
def clean(df_1):
    
    df_1['departure_time'] = pd.to_datetime(df_1['departure_time'], format='%I:%M%p').dt.time
    df_1['arrival_time'] = pd.to_datetime(df_1['arrival_time'], format='%I:%M%p').dt.time

    df_1['airfare'] = df_1['airfare'].replace({'\$':'','\,':''}, regex=True)
    df_1['airfare'] = pd.to_numeric(df_1['airfare'])

    df_1['hours'] = df_1['duration'].str.split("h", n = 1, expand = True) [0]
    df_1['minutes'] = df_1['duration'].str.split(" ", n = 1, expand = True) [1]
    df_1['minutes'] = df_1['minutes'].str.split("m", n = 1, expand = True) [0]

    df_1['hours'] = pd.to_numeric(df_1['hours'])
    df_1['minutes'] = pd.to_numeric(df_1['minutes'])

    df_1['duration'] = df_1['hours'] + (df_1['minutes']/60)
    df_1['duration'] = df_1['duration'].apply(lambda x: round(x, 1))

    df_1.drop(['hours', 'minutes'], axis=1, inplace=True)
    df_1['num_stops'] = df_1['num_stops'].replace({' stops':'',' stop':''}, regex=True)
    df_1['num_stops'] = df_1['num_stops'].replace({'\(':'','\)':''}, regex=True)
    df_1['num_stops'] = pd.to_numeric(df_1['num_stops'])
    
    return df_1
    
# REMOVE OUTLIERS
def remove_outliers(df_1):
    
    # For each departure date
    dates = df_1['departure_date'].unique()
    df_2 = pd.DataFrame()
    
    for d in dates:
        temp_df = df_1[df_1['departure_date'] == d]
   
        # Airfare
        q1 = temp_df['airfare'].quantile(0.25)
        q3 = temp_df['airfare'].quantile(0.5)

        iqr = q3-q1
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr

        temp_df = temp_df.loc[(temp_df['airfare'] > fence_low) & (temp_df['airfare'] < fence_high)]
        
        df_2 = df_2.append(temp_df)    
        
    # Duration
    duration_limit = 1.5*df_2['duration'].mean()
    df_3 = df_2.loc[(df_2['duration'] < duration_limit)]

    return df_3

# Combine data
path = path = r'/Users/kittysu/Documents/Projects/PyFly/Flight_Data' 
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, error_bad_lines=False)
    
    df_temp_1 = clean(df)
    df_temp_2 = remove_outliers(df_temp_1)
    
    li.append(df_temp_2)

master_df_1 = pd.concat(li, axis=0, ignore_index=True)
master_df_1.dropna(inplace=True)

master_df_1.tail()

b'Skipping line 1281: expected 8 fields, saw 15\n'


,departure_time,arrival_time,airfare,airline,duration,num_stops,departure_date,days_to_departure
31630,11:55:00,06:35:00,1103,Delta,28.7,2,04/15/2019,31
31631,12:03:00,07:30:00,1103,Delta,29.4,2,04/15/2019,31
31632,11:55:00,07:30:00,1103,Delta,29.6,2,04/15/2019,31
31633,10:30:00,06:35:00,1103,Delta,30.1,2,04/15/2019,31
31634,10:30:00,06:35:00,1103,Delta,30.1,2,04/15/2019,31


In [10]:
# ADD DATA
master_df_2 = master_df_1.copy()

# DAY OF THE WEEK
master_df_2['departure_date'] = pd.to_datetime(master_df_2['departure_date'])
master_df_2['day_of_week'] = master_df_2['departure_date'].dt.dayofweek

#days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
#master_df_3['day_of_week'] = master_df_3['day_of_week'].apply(lambda x: days[x])

# TIME OF DAY
conditions_1 = [(master_df_2['departure_time'] >= dt.time(hour=6)) & (master_df_2['departure_time'] < dt.time(hour=12)),
                (master_df_2['departure_time'] >= dt.time(hour=12)) & (master_df_2['departure_time'] < dt.time(hour=21)),
                (master_df_2['departure_time'] >= dt.time(hour=21)), (master_df_2['departure_time'] < dt.time(hour=6))]

conditions_2 = [(master_df_2['arrival_time'] >= dt.time(hour=6)) & (master_df_2['arrival_time'] < dt.time(hour=12)),
                (master_df_2['arrival_time'] >= dt.time(hour=12)) & (master_df_2['arrival_time'] < dt.time(hour=21)),
                (master_df_2['arrival_time'] >= dt.time(hour=21)), (master_df_2['arrival_time'] < dt.time(hour=6))]


#choices = ['morning', 'evening', 'night', 'night']
choices = ['0', '1', '2', '2']
master_df_2['departure_category'] = np.select(conditions_1, choices, default='')
master_df_2['arrival_category'] = np.select(conditions_2, choices, default='')

master_df_2.tail()

,departure_time,arrival_time,airfare,airline,duration,num_stops,departure_date,days_to_departure,day_of_week,departure_category,arrival_category
31630,11:55:00,06:35:00,1103,Delta,28.7,2,2019-04-15,31,0,0,0
31631,12:03:00,07:30:00,1103,Delta,29.4,2,2019-04-15,31,0,1,0
31632,11:55:00,07:30:00,1103,Delta,29.6,2,2019-04-15,31,0,0,0
31633,10:30:00,06:35:00,1103,Delta,30.1,2,2019-04-15,31,0,0,0
31634,10:30:00,06:35:00,1103,Delta,30.1,2,2019-04-15,31,0,0,0


In [11]:
# Save data
export_csv = master_df_2.to_csv (r'/Users/kittysu/Documents/Projects/PyFly/non_encoded_data.csv', index = None, header=True) 
print("done")

done


In [12]:
# ENCODE DATA
master_df_3 = master_df_2.copy()

# AIRLINE
airline_df = pd.read_csv(r'/Users/kittysu/Documents/Projects/PyFly/airline.csv', index_col=None, header=0)

airline_df.set_index('airline', inplace=True)

master_df_3.set_index('airline', inplace=True)
master_df_3 = pd.merge(master_df_3,airline_df, how='left', left_index=True, right_index=True)

master_df_3.reset_index(drop=True, inplace=True)
master_df_3.rename(columns={'encode':'airline'}, inplace=True)

# DATE
master_df_3['day_of_year'] = ''
for i in range(0, len(master_df_3)):
    
    departure_date = master_df_3.iloc[i]['departure_date']
    master_df_3.at[i,'day_of_year'] = departure_date.timetuple().tm_yday

master_df_3.tail()

,departure_time,arrival_time,airfare,duration,num_stops,departure_date,days_to_departure,day_of_week,departure_category,arrival_category,airline,day_of_year
31630,16:20:00,08:35:00,975,26.2,2,2019-04-15,31,0,1,0,22,105
31631,16:20:00,09:10:00,975,26.8,2,2019-04-15,31,0,1,0,22,105
31632,16:20:00,09:40:00,975,27.3,2,2019-04-15,31,0,1,0,22,105
31633,16:20:00,10:40:00,975,28.3,2,2019-04-15,31,0,1,0,22,105
31634,16:20:00,11:40:00,975,29.3,2,2019-04-15,31,0,1,0,22,105


In [13]:
# Save data
master_df_3.drop('departure_time', axis=1, inplace=True)
master_df_3.drop('arrival_time', axis=1, inplace=True)

master_df_3.head()

export_csv = master_df_3.to_csv (r'/Users/kittysu/Documents/Projects/PyFly/encoded_data.csv', index = None, header=True) 

print("done")

done
